In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools.render import render_text_description
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
import json

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [5]:
@tool
def count_emails(last_n_days: int) -> int:
    """Multiply two integers together."""
    return last_n_days * 2


@tool
def send_email(message: str, recipient: str) -> str:
    "Add two integers."
    return f"Successfully sent email:\n{message}\n to {recipient}."

In [6]:
tools=[count_emails,send_email]
llmWithTools=llm.bind_tools(tools=tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [7]:
def toolIdentifier(msg: AIMessage) -> Runnable:
    toolMap={tool.name:tool for tool in tools}
    for toolInfo in msg.tool_calls:
        toolInfo['output']=toolMap[toolInfo['name']].invoke(input=toolInfo['args'])
    return msg.tool_calls

In [8]:
chain=llmWithTools|toolIdentifier

In [9]:
question1="How many emails did I get in the last 5 days?"

In [10]:
chain.invoke(input=question1)

{'name': 'count_emails', 'description': 'Multiply two integers together.', 'parameters': {'type_': 6, 'description': 'Multiply two integers together.', 'properties': {'last_n_days': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['last_n_days'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'send_email', 'description': 'Add two integers.', 'parameters': {'type_': 6, 'description': 'Add two integers.', 'properties': {'recipient': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'message': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['message', 'recipient'], 'format_': '', 'nullable': False, 'enum': [], 'max_items':

[{'name': 'count_emails',
  'args': {'last_n_days': 5.0},
  'id': '14c3d13e-3100-496c-ab10-bc79b81eaaed',
  'type': 'tool_call',
  'output': 10}]

In [11]:
question2="Send an Email to Mr. Ritish Adhikari stating that you will be late?"

In [12]:
chain.invoke(input=question2)

{'name': 'count_emails', 'description': 'Multiply two integers together.', 'parameters': {'type_': 6, 'description': 'Multiply two integers together.', 'properties': {'last_n_days': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['last_n_days'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'send_email', 'description': 'Add two integers.', 'parameters': {'type_': 6, 'description': 'Add two integers.', 'properties': {'recipient': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'message': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['message', 'recipient'], 'format_': '', 'nullable': False, 'enum': [], 'max_items':

[{'name': 'send_email',
  'args': {'recipient': 'Mr. Ritish Adhikari', 'message': 'I will be late'},
  'id': '6f9d79d7-5578-4fed-8625-2b6534e1d781',
  'type': 'tool_call',
  'output': 'Successfully sent email:\nI will be late\n to Mr. Ritish Adhikari.'}]

<h3> Adding of Human Approval </h3>

In [13]:
toolCall=llmWithTools.invoke(input=question2).tool_calls[0]
toolCall

{'name': 'count_emails', 'description': 'Multiply two integers together.', 'parameters': {'type_': 6, 'description': 'Multiply two integers together.', 'properties': {'last_n_days': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['last_n_days'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'send_email', 'description': 'Add two integers.', 'parameters': {'type_': 6, 'description': 'Add two integers.', 'properties': {'recipient': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'message': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['message', 'recipient'], 'format_': '', 'nullable': False, 'enum': [], 'max_items':

{'name': 'send_email',
 'args': {'recipient': 'Mr. Ritish Adhikari', 'message': 'I will be late'},
 'id': '2ab31d5f-7876-4e10-b8fd-9fdb8ef2e876',
 'type': 'tool_call'}

In [14]:
print(json.dumps(obj=toolCall,indent=2))

{
  "name": "send_email",
  "args": {
    "recipient": "Mr. Ritish Adhikari",
    "message": "I will be late"
  },
  "id": "2ab31d5f-7876-4e10-b8fd-9fdb8ef2e876",
  "type": "tool_call"
}


In [15]:
print("".join(json.dumps(obj=toolcall, indent=2) for toolcall in llmWithTools.invoke(input=question2).tool_calls))

{'name': 'count_emails', 'description': 'Multiply two integers together.', 'parameters': {'type_': 6, 'description': 'Multiply two integers together.', 'properties': {'last_n_days': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['last_n_days'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'send_email', 'description': 'Add two integers.', 'parameters': {'type_': 6, 'description': 'Add two integers.', 'properties': {'recipient': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'message': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['message', 'recipient'], 'format_': '', 'nullable': False, 'enum': [], 'max_items':

In [16]:
def humanApproval(msg: AIMessage) -> Runnable:
    toolStrs="".join(json.dumps(obj=toolcall, indent=2) for toolcall in msg.tool_calls)

    inputMessage=(
        f"Do you approve of the following tool invocations \n\n {toolStrs}\n\n"
        "Anything except 'Y'/'Yes' (case-insensitive) will be treated as a No")

    resp=input(inputMessage)
    if resp.lower() not in ['y','yes']:
        raise ValueError(f"Tool Invocation is Not approved: \n\n {toolStrs}")
    else:
        return msg

In [17]:
chain=llmWithTools|humanApproval|toolIdentifier

In [19]:
chain.invoke(input=question2)

{'name': 'count_emails', 'description': 'Multiply two integers together.', 'parameters': {'type_': 6, 'description': 'Multiply two integers together.', 'properties': {'last_n_days': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['last_n_days'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'send_email', 'description': 'Add two integers.', 'parameters': {'type_': 6, 'description': 'Add two integers.', 'properties': {'recipient': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'message': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['message', 'recipient'], 'format_': '', 'nullable': False, 'enum': [], 'max_items':

Do you approve of the following tool invocations 

 {
  "name": "send_email",
  "args": {
    "recipient": "Mr. Ritish Adhikari",
    "message": "I will be late"
  },
  "id": "68e4a569-f964-4f3a-9458-2ca77c437b6e",
  "type": "tool_call"
}

Anything except 'Y'/'Yes' (case-insensitive) will be treated as a No N


ValueError: Tool Invocation is Not approved: 

 {
  "name": "send_email",
  "args": {
    "recipient": "Mr. Ritish Adhikari",
    "message": "I will be late"
  },
  "id": "68e4a569-f964-4f3a-9458-2ca77c437b6e",
  "type": "tool_call"
}

In [20]:
chain.invoke(input=question1)

{'name': 'count_emails', 'description': 'Multiply two integers together.', 'parameters': {'type_': 6, 'description': 'Multiply two integers together.', 'properties': {'last_n_days': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['last_n_days'], 'format_': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0'}}
{'name': 'send_email', 'description': 'Add two integers.', 'parameters': {'type_': 6, 'description': 'Add two integers.', 'properties': {'recipient': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'message': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['message', 'recipient'], 'format_': '', 'nullable': False, 'enum': [], 'max_items':

Do you approve of the following tool invocations 

 {
  "name": "count_emails",
  "args": {
    "last_n_days": 5.0
  },
  "id": "b91fcfb9-a64f-4fba-a521-6fb0b373d5a4",
  "type": "tool_call"
}

Anything except 'Y'/'Yes' (case-insensitive) will be treated as a No Y


[{'name': 'count_emails',
  'args': {'last_n_days': 5.0},
  'id': 'b91fcfb9-a64f-4fba-a521-6fb0b373d5a4',
  'type': 'tool_call',
  'output': 10}]

In [20]:
chain.invoke(input=question1)

Do you approve of the following tool invocations 

 {
  "name": "count_emails",
  "args": {
    "last_n_days": 5.0
  },
  "id": "50f30814-be8d-44c2-9bf3-c5282a292081",
  "type": "tool_call"
}

Anything except 'Y'/'Yes' (case-insensitive) will be treated as a No n


ValueError: Tool Invocation is Not approved: 

 {
  "name": "count_emails",
  "args": {
    "last_n_days": 5.0
  },
  "id": "50f30814-be8d-44c2-9bf3-c5282a292081",
  "type": "tool_call"
}